In [1]:
import random
from typing import TypedDict
import statistics
import numpy as np

In [2]:
PET_LEVEL_TO_DROPRATE_MULTIPLIER = {
    0:1.0,
    1:1.0,
    2:1.0,
    3:1.0,
    4:1.33,
    5:1.67,
    6:2.0,
    7:2.33,
    8:2.67,
    9:3.0,
    10:3.33,
    11:3.67,
    12:4.0,
}

class PetDropRatesDict(TypedDict):
    item: str
    dropRate: int


class Pet:
    def __init__(self, name:str, drops_dict: PetDropRatesDict, cooldown: int):
        self.name = name
        self.drops = drops_dict
        self.cooldown = cooldown


In [3]:
classic_dragon_pet_drops: PetDropRatesDict = {
    "Hoard":100,
    "Site Card":150,
    "Event Card":500
}
classic_dragon_pet = Pet("Classic Dragon", classic_dragon_pet_drops, cooldown=23)

black_dragon_pet_drops: PetDropRatesDict = {
    "Hoard":100,
    "Site Card":150,
    "Event Card":450
}
black_dragon_pet = Pet("Black Dragon", black_dragon_pet_drops, cooldown=23)

ada_dwarf_drops: PetDropRatesDict = {
    "Adamantium Ore":200,
}
ada_dwarf_pet = Pet("Adamantium Dwarf", ada_dwarf_drops, cooldown=24)



In [4]:
def run_simulation(sim_days: int, pet:Pet, pet_level:int):
    simulation_steps = sim_days*24
    pet_level_drop_rate_modifier = PET_LEVEL_TO_DROPRATE_MULTIPLIER[pet_level]
    
    for key in pet.drops.keys():
        chance_to_drop = pet_level_drop_rate_modifier/pet.drops.get(key)
        #print(f'Chance to drop {key} is {pet_level_drop_rate_modifier}/{pet.drops.get(key)} = {pet_level_drop_rate_modifier/pet.drops.get(key)} ')

    drop_results = dict() # stores list of total days since last drop per each item
    steps_since_last_drop = dict() # stores temporary days since last drop count for each item 
    for drop_item in pet.drops.keys():
        drop_results[drop_item] = list()
        steps_since_last_drop[drop_item] = 0

    last_drop_step = 0
    for sim_step in range(simulation_steps):
        if((sim_step - last_drop_step) < pet.cooldown):
            #print(f'Sim Step is  {sim_step}, last item drop is {last_drop_step} continuing!')
            for drop_item in pet.drops.keys(): #increase steps since last drop for each drop item
                steps_since_last_drop[drop_item] += 1
            continue #move to next sim step

        for drop_item in pet.drops.keys():
            chance_to_drop = pet_level_drop_rate_modifier/pet.drops.get(drop_item)
            item_dropped = random.random() < chance_to_drop
            if(item_dropped):
                #print(f'Item {drop_item} dropped after {steps_since_last_drop[drop_item]} steps')
                drop_results[drop_item].append(steps_since_last_drop[drop_item]+1) # add one more step to account for this step
                steps_since_last_drop[drop_item] = 0 # reset steps since the last time this item dropped
                last_drop_step = sim_step # set the global last time any item dropped
            else: # no item drop, add step to its count since last drop
                steps_since_last_drop[drop_item] += 1
    
    return drop_results
            

In [5]:
def run_n_sim(num_sims:int, years_per_sim:int, pet:Pet, pet_level:int):
    all_results = dict()
    #combine drop time results from all simulations
    for n in range(num_sims):
        for drop_item in pet.drops.keys():
            all_results[drop_item] = list()
        
        drop_results = run_simulation(years_per_sim*365, pet, pet_level)
        for drop_item in pet.drops.keys():
            all_results[drop_item].extend(drop_results.get(drop_item))
    return all_results

In [6]:
def print_n_sims_result(pet:Pet, pet_level, all_results):
    # simulation output is in steps, convert steps to days while printing stats 
    for drop_item in pet.drops.keys():
        avg_drop_days = np.mean(all_results.get(drop_item))/24
        stddev_drop_days = np.std(all_results.get(drop_item))/24
        min_drop_days = np.min(all_results.get(drop_item))/24
        max_drop_days = np.max(all_results.get(drop_item))/24
        median_drop_days = np.median(all_results.get(drop_item))/24
        percentile95_drop_days = np.percentile(all_results.get(drop_item),95)/24

        drop_string = f'{pet.name} @ Level  {pet_level} - stats [days] for {drop_item}:\n ' + \
                    f'Avg: {round(avg_drop_days,2)} StdDev: {round(stddev_drop_days,2)} ' + \
                    f'Median: {round(median_drop_days,2)} 95%ile: {round(percentile95_drop_days,2)} '+ \
                    f'Min: {round(min_drop_days,2)} Max: {round(max_drop_days,2)} '
        print(drop_string)

In [7]:
NUM_SIMS = 10000
SIM_YEARS = 30
PET = classic_dragon_pet

for level in range(12,13):
    level_results = run_n_sim(NUM_SIMS, SIM_YEARS, PET, level)
    print_n_sims_result(PET, level, level_results)
    print()

Classic Dragon @ Level  12 - stats [days] for Hoard:
 Avg: 2.69 StdDev: 1.94 Median: 2.12 95%ile: 6.54 Min: 0.96 Max: 20.25 
Classic Dragon @ Level  12 - stats [days] for Site Card:
 Avg: 4.08 StdDev: 3.43 Median: 3.04 95%ile: 10.92 Min: 0.96 Max: 30.62 
Classic Dragon @ Level  12 - stats [days] for Event Card:
 Avg: 13.31 StdDev: 12.6 Median: 9.54 95%ile: 37.25 Min: 0.96 Max: 107.5 



In [8]:
NUM_SIMS = 10000
SIM_YEARS = 30
SIM_PET_DROPS = black_dragon_pet

for level in range(12,13):
    level_results = run_n_sim(NUM_SIMS, SIM_YEARS, SIM_PET_DROPS, level)
    print_n_sims_result(SIM_PET_DROPS, level, level_results)
    print()

Black Dragon @ Level  12 - stats [days] for Hoard:
 Avg: 2.68 StdDev: 1.93 Median: 2.12 95%ile: 6.5 Min: 0.96 Max: 20.0 
Black Dragon @ Level  12 - stats [days] for Site Card:
 Avg: 4.12 StdDev: 3.4 Median: 3.08 95%ile: 10.96 Min: 0.96 Max: 29.25 
Black Dragon @ Level  12 - stats [days] for Event Card:
 Avg: 12.4 StdDev: 11.62 Median: 8.83 95%ile: 35.58 Min: 0.96 Max: 77.42 

